<a href="https://colab.research.google.com/github/SiEggo/scraping/blob/main/basic_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.petsecure.com.au/cover-choices/"

respons = rs.get(url)
print(respons)

<Response [200]>


In [ ]:
url = "https://quotes.toscrape.com/page/1/"
respons = requests.get(url)

soup = BeautifulSoup(respons)

In [ ]:
import pandas as pd
import duckdb as ddb
from bs4 import BeautifulSoup as bs

con = ddb.connect("test_scraping")

In [ ]:
soup = bs(respons.content, 'html.parser')

In [ ]:
assurance = soup.find('div',{'class' : 'table-2'})

In [ ]:
table = assurance.find('table')

In [ ]:
header = []
for th in table.find_all('th'):
    header.append(th.text.strip())
    print(th.text.strip())
    print("done")

header

Benefits
done

done
Accident
& Illness Cover
done


['Benefits', '', 'Accident\n& Illness Cover']

In [ ]:
row = []
for tr in table.find_all('tr'):
    cleansing = []
    for td in tr.find_all('td'):
        cleansing.append(td.text.strip())
    if len(cleansing) > 0:
        row.append(cleansing)

In [ ]:
row

[['', '', ''],
 ['$12,000 annual benefit limit', '', ''],
 ['Limit on individual claims', '', 'Up to the annual limit – $12,000.'],
 ['Cruciate ligament conditions', '', '$2,600 per annum*'],
 ['Consultations', '', '$300 per annum'],
 ['Lifetime cover',
  '',
  'Yes. As long as the Policy is renewed annually without any break in cover, subject to the terms and conditions of your renewal policy'],
 ['Chronic conditions that require ongoing treatment continually covered',
  '',
  ''],
 ['Emergency boarding fees for the insured pet if you, the policyholder, are hospitalised for more than four days.',
  '',
  '$500 per annum'],
 ['Tick Paralysis Treatment', '', '$500 included'],
 ['Multi pet discount\n(maximum of one 10% discount per policy)',
  '',
  '10% discount'],
 ['Service dog discount\n(maximum of one 10% discount per policy)',
  '',
  '10% discount'],
 ['Australian Pension Cardholder discount\n(maximum of one 10% discount per policy)',
  '',
  '10% discount'],
 ['Rescue pet discoun

In [ ]:
df = pd.DataFrame(row, columns=header)
df

,Benefits,,Accident\n& Illness Cover
0,,,
1,"$12,000 annual benefit limit",,
2,Limit on individual claims,,"Up to the annual limit – $12,000."
3,Cruciate ligament conditions,,"$2,600 per annum*"
4,Consultations,,$300 per annum
5,Lifetime cover,,Yes. As long as the Policy is renewed annually...
6,Chronic conditions that require ongoing treatm...,,
7,Emergency boarding fees for the insured pet if...,,$500 per annum
8,Tick Paralysis Treatment,,$500 included
9,Multi pet discount\n(maximum of one 10% discou...,,10% discount


In [ ]:
df = df[df['Benefits'] != '']
df

,Benefits,,Accident\n& Illness Cover
1,"$12,000 annual benefit limit",,
2,Limit on individual claims,,"Up to the annual limit – $12,000."
3,Cruciate ligament conditions,,"$2,600 per annum*"
4,Consultations,,$300 per annum
5,Lifetime cover,,Yes. As long as the Policy is renewed annually...
6,Chronic conditions that require ongoing treatm...,,
7,Emergency boarding fees for the insured pet if...,,$500 per annum
8,Tick Paralysis Treatment,,$500 included
9,Multi pet discount\n(maximum of one 10% discou...,,10% discount
10,Service dog discount\n(maximum of one 10% disc...,,10% discount


In [ ]:
df.reset_index(drop=True)

,Benefits,,Accident\n& Illness Cover
0,"$12,000 annual benefit limit",,
1,Limit on individual claims,,"Up to the annual limit – $12,000."
2,Cruciate ligament conditions,,"$2,600 per annum*"
3,Consultations,,$300 per annum
4,Lifetime cover,,Yes. As long as the Policy is renewed annually...
5,Chronic conditions that require ongoing treatm...,,
6,Emergency boarding fees for the insured pet if...,,$500 per annum
7,Tick Paralysis Treatment,,$500 included
8,Multi pet discount\n(maximum of one 10% discou...,,10% discount
9,Service dog discount\n(maximum of one 10% disc...,,10% discount


In [ ]:
df.to_excel("result_scraping.xlsx",index=False)